In [3]:
import numpy as np
import math
import pandas as pd

! pip3 install pyyaml
!pip install altair_saver
import yaml
from yaml.loader import SafeLoader
import altair as alt

In [4]:
with open('config.yml') as f:
        data = yaml.load(f, Loader=SafeLoader)

In [5]:
casos=pd.read_json(data['url'])

In [6]:
print(data['url'])

https://integrasus.saude.ce.gov.br/api/coronavirus/curva-epidemiologica?dataInicio=2020-01-01&dataFim=2021-07-12&minDate=2020-01-01&maxDate=2021-07-12&tipo=Confirmados&idMunicipio=&idRegiaoSaude=&idMacrorregiao=&casosHospitalizados=false&casosProfissionais=false&casosIndigenas=false&casosEstudante=false


In [7]:
casos

,quantidade,quantidadeSuspeitos,mediaMovel,data
0,242,9,0.00,15/03/2020
1,170,5,0.00,16/03/2020
2,196,2,0.00,17/03/2020
3,257,7,0.00,18/03/2020
4,224,5,0.00,19/03/2020
...,...,...,...,...
480,315,511,481.00,08/07/2021
481,267,374,429.42,09/07/2021
482,65,256,361.28,10/07/2021
483,27,149,242.57,11/07/2021


In [35]:

def SIRV_model(vaccine_rate_1000=0.002,V0=0,N =6587940,I0 = 16089,vaccine_eff=0.50,mortality_rate=1.9e-2,percentual_infectados=0.001,day_interval=30):
    import yaml
    import numpy as np
    import math
    import pandas as pd
    from yaml.loader import SafeLoader
    import pandas as pd
    from datetime import  datetime
    from datetime import timedelta
    import altair as alt



    # Open the file and load the file
    with open('config.yml') as f:
        data = yaml.load(f, Loader=SafeLoader)
    
    casos=pd.read_json(data['url'])
    
    casos['quantidade_nao_normalizada']=casos.quantidade.values
    
    total_infectados=sum(casos.quantidade_nao_normalizada.values)
    
    N=8843000-total_infectados
    
    casos.quantidade=casos.quantidade/N
    

    
    
    
    #casos.data=pd.to_datetime(casos.data)

   
    
    print(casos[-5:]['data'].values[0])
    data_inicial=casos[-5:]['data'].values[0]
    infectados=casos[-5:]['quantidade_nao_normalizada'].values[0]
    
    
    
    I0=infectados


    
    rate_of_removal=7.81e-2
    rate_of_infection= 9.76e-2
    alpha=vaccine_eff
    beta =rate_of_infection
    gamma =rate_of_removal
    mu=mortality_rate
    R0 = 0
    

   
    dt=1
    print(data_inicial)

    start_date=  datetime.strptime( data_inicial, '%d/%m/%Y')
    end_date=  datetime.strptime('Jun 8 2021', '%b %d %Y')
    prediction_end_date=datetime.strptime('Jan 30 2023', '%b %d %Y')

    tdays=(prediction_end_date-start_date).days
    
    T=tdays
    
    vaccine_rate=vaccine_rate_1000
    I0=I0/N 
    R0=R0/N


    
    def vaccine_pulse(vaccine_number,days):

        u=[ vaccine_number for i in range(0,days)]
        return u

    u=vaccine_pulse(vaccine_rate,tdays)
    S = [0 for i in range(0,math.ceil(T/dt))]
    R = [0 for i in range(0,math.ceil(T/dt))]
    I = [0 for i in range(0,math.ceil(T/dt))]
    V = [0 for i in range(0,math.ceil(T/dt))]
    
    S[0] =1-I0-R0-V0 
    I[0] = I0
    R[0]=R0
    V[0]=V0

    for t in range(0,math.ceil(T/dt)-1):
        #Boundary control
        if S[t]<=0:
            u[t]=0
            S[t]=0
        
        # Equations of the model
        dS = (-beta*I[t]*(S[t]/(1+0.2*S[t]))-alpha*u[t]) * dt
        dI = (beta*I[t]*(S[t]/(1+0.2*S[t])) - gamma*I[t]) * dt
        dR = (gamma*I[t]) * dt
        dV=alpha*u[t] * dt
        
        r = t % day_interval

        if r==0 and S[t]>0:
            
            I[t+1] = I[t] + dI+percentual_infectados
            S[t+1] = S[t] + dS
          
        else:
            S[t+1] = S[t] + dS
            I[t+1] = I[t] + dI  
        
        
        R[t+1] = R[t] + dR
        V[t+1] = V[t] + dV

    date_list = [timedelta(days=x)+start_date for x in range(tdays)]    
    removidos=[r for r in R]    
    vacinados=[v for v in V]        
    sucetiveis=[s for s in S]
    infectados=[i for i in I]
    import pandas as pd
    df=pd.DataFrame(columns=['data','infectados','sucetiveis','removidos','vacinados'])
    df['data']=date_list
    df['infectados']=infectados
    df['sucetiveis']=sucetiveis
    df['removidos']=removidos

    df['vacinados']=vacinados
    df['infectados_acumulados'] = df['infectados'].cumsum()
    
    df['vacinados_acumulados'] = df['vacinados'].cumsum()
    casos.data=[ datetime.strptime(d, '%d/%m/%Y') for d in casos.data.values]   
    casos['infectados_acumulados']=casos.quantidade.cumsum()
    
    maximo_quantidade=np.max(casos.infectados_acumulados)
    
    print('maximo',maximo_quantidade)
    
    df['infectados_acumulados'] =df['infectados_acumulados']+maximo_quantidade
    
    
    casos_anteriores=alt.Chart(casos)\
        .mark_line()\
        .encode(x='data',y='quantidade')\
        .properties(
            width=600,
    title=alt.TitleParams(
        ['','Gráfico com o Número de infectados por Covid e projeção com o modelo SIRV.'],
        baseline='bottom',
        orient='bottom',
        anchor='end',
        fontWeight='normal',
        fontSize=10
    ))


    casos_acumulados=alt.Chart(casos)\
        .mark_line()\
        .encode(x='data',y='infectados_acumulados')\
        .properties(
            width=600,
    title=alt.TitleParams(
        ['','Gráfico com o Número de infectados por Covid e projeção com o modelo SIRV.'],
        baseline='bottom',
        orient='bottom',
        anchor='end',
        fontWeight='normal',
        fontSize=10
    ))

    novos_casos=alt.Chart(df)\
        .mark_line()\
        .encode(x='data',y='infectados',color=alt.value('red'))\
        .properties(
            width=600,
    title=alt.TitleParams(
        ['','','Projeção dos casos de Covid de acordo com o modelo SIRV.'],
        baseline='bottom',
        orient='bottom',
        anchor='end',
        fontWeight='normal',
        fontSize=10
    ))


    novos_casos_acumulados=alt.Chart(df)\
        .mark_line()\
        .encode(x='data',y='infectados_acumulados',color=alt.value('red'))\
        .properties(
            width=600,
    title=alt.TitleParams(
        ['','','Projeção dos casos acumulados de Covid de acordo com o modelo SIRV.'],
        baseline='bottom',
        orient='bottom',
        anchor='end',
        fontWeight='normal',
        fontSize=10
    ))

    vacinados_acumulados_casos_acumulados=alt.Chart(df)\
        .mark_line()\
        .encode(x='data',y='vacinados',color=alt.value('red'))\
        .properties(
            width=600,
    title=alt.TitleParams(
        ['','','Projeção dos vacinados acumulados de Covid de acordo com o modelo SIRV.'],
        baseline='bottom',
        orient='bottom',
        anchor='end',
        fontWeight='normal',
        fontSize=10
    ))
    grafico=alt.vconcat(casos_anteriores+novos_casos,novos_casos,casos_acumulados+novos_casos_acumulados,vacinados_acumulados_casos_acumulados)
    grafico.save('chart.html')
    
    
    
    return grafico,df

In [39]:
grafico,df=SIRV_model(vaccine_rate_1000=0.002,vaccine_eff=1,percentual_infectados=0)


08/07/2021
08/07/2021
maximo 0.11352320606034651


In [40]:
df

,data,infectados,sucetiveis,removidos,vacinados,infectados_acumulados,vacinados_acumulados
0,2021-07-08,3.966525e-05,0.999960,0.000000,0.000,0.113563,0.000
1,2021-07-09,3.979339e-05,0.997957,0.000003,0.002,0.113603,0.002
2,2021-07-10,3.991655e-05,0.995954,0.000006,0.004,0.113643,0.006
3,2021-07-11,4.003466e-05,0.993951,0.000009,0.006,0.113683,0.012
4,2021-07-12,4.014767e-05,0.991947,0.000012,0.008,0.113723,0.020
...,...,...,...,...,...,...,...
566,2023-01-25,3.065852e-15,0.000000,0.000416,1.000,0.118850,316.500
567,2023-01-26,2.826409e-15,0.000000,0.000416,1.000,0.118850,317.500
568,2023-01-27,2.605666e-15,0.000000,0.000416,1.000,0.118850,318.500
569,2023-01-28,2.402163e-15,0.000000,0.000416,1.000,0.118850,319.500


In [41]:
grafico

alt.VConcatChart(...)

In [59]:
alt.vconcat(alt.Chart(casos).mark_line().encode(
x='data'
,y='quantidade')+alt.Chart(df).mark_line().encode(
x='data'
,y='infectados') , alt.Chart(df).mark_line().encode(
x='data'
,y='infectados'))

alt.VConcatChart(...)

In [44]:
df

,data,infectados,sucetiveis,removidos,vacinados
0,2021-07-08,256.000000,6.587684e+06,0.000000e+00,0.00
1,2021-07-09,6844.767059,6.584369e+06,1.999360e+01,3293.97
2,2021-07-10,6866.646995,6.580519e+06,5.545699e+02,6587.94
3,2021-07-11,6888.324782,6.576667e+06,1.090855e+03,9881.91
4,2021-07-12,6909.797896,6.572813e+06,1.628833e+03,13175.88
...,...,...,...,...,...
882,2023-12-07,6117.803136,2.787960e+06,1.086218e+06,2905281.54
883,2023-12-08,5872.971287,2.784433e+06,1.086696e+06,2908575.51
884,2023-12-09,5637.676630,2.780916e+06,1.087155e+06,2911869.48
885,2023-12-10,5411.559026,2.777408e+06,1.087595e+06,2915163.45


In [45]:
import altair as alt
alt.Chart(df).mark_line().encode(
x='data'
,y='infectados')

alt.Chart(...)

In [38]:
df=SIRV_model(day_interval=10,percentual_infectados=0,vaccine_eff=0.5)

08/07/2021


In [39]:
import altair as alt
alt.Chart(df).mark_line().encode(
x='data'
,y='infectados')

alt.Chart(...)

In [251]:
df=SIRV_model(day_interval=10,percentual_infectados=0,vaccine_eff=0.5,vaccine_rate_1000=0.8)

In [252]:
df

,data,infectados,sucetiveis,removidos,vacinados
0,2021-06-08,16089.000000,6.571851e+06,0.000000,0.000
1,2021-06-09,16138.356865,6.567910e+06,1256.550900,2635.176
2,2021-06-10,16187.210191,6.563965e+06,2516.956571,5270.352
3,2021-06-11,16235.553779,6.560018e+06,3781.177687,7905.528
4,2021-06-12,16283.381474,6.556067e+06,5049.174437,10540.704
...,...,...,...,...,...
912,2023-12-07,0.084601,3.804043e+06,380616.776412,2403280.512
913,2023-12-08,0.082268,3.801407e+06,380616.783019,2405915.688
914,2023-12-09,0.079997,3.798772e+06,380616.789444,2408550.864
915,2023-12-10,0.077786,3.796137e+06,380616.795692,2411186.040


In [253]:
import altair as alt
alt.Chart(df).mark_line().encode(
x='data'
,y='infectados')

alt.Chart(...)

In [254]:
import altair as alt
alt.Chart(df).mark_line().encode(
x='data'
,y='vacinados')

alt.Chart(...)